In [1]:
import torch

In [2]:
input = torch.ones([2, 2], requires_grad=False)
w1 = torch.tensor(2.0, requires_grad=True)
w2 = torch.tensor(3.0, requires_grad=True)
w3 = torch.tensor(4.0, requires_grad=True)

l1 = input * w1
l2 = l1 + w2
l3 = l1 * w3
l4 = l2 * l3
loss = l4.mean()

l1.retain_grad()
l4.retain_grad()
loss.retain_grad()

loss.backward()

print(loss.grad)
print(l4.grad)
print(l1.grad)

print(w1.data, w1.grad, w1.grad_fn)

print(l1.data, l1.grad, l1.grad_fn)

print(loss.data, loss.grad, loss.grad_fn)

tensor(1.)
tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])
tensor([[7., 7.],
        [7., 7.]])
tensor(2.) tensor(28.) None
tensor([[2., 2.],
        [2., 2.]]) tensor([[7., 7.],
        [7., 7.]]) <MulBackward0 object at 0x7fcf07475950>
tensor(40.) tensor(1.) <MeanBackward0 object at 0x7fcf07475a10>


## NET with torch

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [2]:
net = Net()

In [3]:
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)


In [4]:
import torch.optim as optim
# 创建一个随机梯度下降（stochastic gradient descent）优化器
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# 创建一个损失函数
criterion = nn.NLLLoss()

In [6]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset
import numpy as np

train = dset.MNIST('processed/training.pt', train=True, transform=transforms.ToTensor(), 
                          target_transform=None, 
                          download=False)


# train_loader = utils.data.DataLoader(train_loader, batch_size=1000, shuffle=True, num_workers=2)
batch_size = 100
train_loader= DataLoader(dataset=train, batch_size=batch_size)

In [8]:
# 运行主训练循环
import matplotlib.pyplot as plt
from torch.autograd import Variable
log_interval = 2

for i, (images, labels) in enumerate(train_loader):        
#     if (i + 1) % 100 == 0:
#         print('batch_number [{}/{}]'.format(i + 1, len(train_loader)))
#         for j in range(len(images)):
#             image = images[j].resize(28, 28) #将(1,28,28)->(28,28)
#             plt.imshow(image)  # 显示图片,接受tensors, numpy arrays, numbers, dicts or lists
#             plt.axis('off')  # 不显示坐标轴
#             plt.title("$The {} picture in {} batch, label={}$".format(j + 1, i + 1, labels[j]))
#             plt.show()
    for j in range(len(images)):
        data=images[j]
        target = labels[j]
        data, target = Variable(data), Variable(target)
        data, target = data.float(), target.long()
        data = data.view(-1, 28*28)
        target = target.view(-1)
        optimizer.zero_grad()

        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()
        if j % log_interval == 0:
            print(f'Train Epoch: {i}, {j}/{len(images)}, Loss: {loss.data:.6f}')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Train Epoch: 0, 0/100, Loss: 2.221058
Train Epoch: 0, 1/100, Loss: 2.344365
Train Epoch: 0, 2/100, Loss: 2.282169
Train Epoch: 0, 3/100, Loss: 2.361927
Train Epoch: 0, 4/100, Loss: 2.360992
Train Epoch: 0, 5/100, Loss: 2.347391
Train Epoch: 0, 6/100, Loss: 2.324021
Train Epoch: 0, 7/100, Loss: 2.466393
Train Epoch: 0, 8/100, Loss: 2.271106
Train Epoch: 0, 9/100, Loss: 2.266706
Train Epoch: 0, 10/100, Loss: 2.363419
Train Epoch: 0, 11/100, Loss: 2.187346
Train Epoch: 0, 12/100, Loss: 2.293019
Train Epoch: 0, 13/100, Loss: 2.440600
Train Epoch: 0, 14/100, Loss: 2.107381
Train Epoch: 0, 15/100, Loss: 2.476936
Train Epoch: 0, 16/100, Loss: 2.374786
Train Epoch: 0, 17/100, Loss: 2.476216
Train Epoch: 0, 18/100, Loss: 2.364106
Train Epoch: 0, 19/100, Loss: 2.452046
Train Epoch: 0, 20/100, Loss: 2.209303
Train Epoch: 0, 21/100, Loss: 2.366051
Train Epoch: 0, 22/100, Loss: 2.404745
Train Epoch: 0, 23/100, Loss: 2.016325
Train Epoch: 0, 24/100, Loss: 2.066274
Train Epoch: 0, 25/100, Loss: 2.336

Train Epoch: 6, 0/100, Loss: 0.284117
Train Epoch: 6, 1/100, Loss: 0.474763
Train Epoch: 6, 2/100, Loss: 1.633468
Train Epoch: 6, 3/100, Loss: 0.031150
Train Epoch: 6, 4/100, Loss: 1.270105
Train Epoch: 6, 5/100, Loss: 0.680987
Train Epoch: 6, 6/100, Loss: 0.060297
Train Epoch: 6, 7/100, Loss: 0.056573
Train Epoch: 6, 8/100, Loss: 0.999268
Train Epoch: 6, 9/100, Loss: 0.063877
Train Epoch: 6, 10/100, Loss: 2.296124
Train Epoch: 6, 11/100, Loss: 1.630586
Train Epoch: 6, 12/100, Loss: 0.007375
Train Epoch: 6, 13/100, Loss: 1.961464
Train Epoch: 6, 14/100, Loss: 8.110122
Train Epoch: 6, 15/100, Loss: 0.487922
Train Epoch: 6, 16/100, Loss: 0.206787
Train Epoch: 6, 17/100, Loss: 0.001981
Train Epoch: 6, 18/100, Loss: 0.621126
Train Epoch: 6, 19/100, Loss: 0.028444
Train Epoch: 6, 20/100, Loss: 2.038465
Train Epoch: 6, 21/100, Loss: 1.444331
Train Epoch: 6, 22/100, Loss: 2.767633
Train Epoch: 6, 23/100, Loss: 0.801422
Train Epoch: 6, 24/100, Loss: 0.014427
Train Epoch: 6, 25/100, Loss: 3.659

Train Epoch: 10, 95/100, Loss: 0.814303
Train Epoch: 10, 96/100, Loss: 2.720411
Train Epoch: 10, 97/100, Loss: 10.473899
Train Epoch: 10, 98/100, Loss: 1.347719
Train Epoch: 10, 99/100, Loss: 0.596741
Train Epoch: 12, 0/100, Loss: 0.000004
Train Epoch: 12, 1/100, Loss: 0.634836
Train Epoch: 12, 2/100, Loss: 0.117212
Train Epoch: 12, 3/100, Loss: 0.055173
Train Epoch: 12, 4/100, Loss: 3.046102
Train Epoch: 12, 5/100, Loss: 0.031085
Train Epoch: 12, 6/100, Loss: 1.546132
Train Epoch: 12, 7/100, Loss: 2.588760
Train Epoch: 12, 8/100, Loss: 0.004120
Train Epoch: 12, 9/100, Loss: 2.871790
Train Epoch: 12, 10/100, Loss: 0.776006
Train Epoch: 12, 11/100, Loss: 0.000117
Train Epoch: 12, 12/100, Loss: 1.346784
Train Epoch: 12, 13/100, Loss: 2.833058
Train Epoch: 12, 14/100, Loss: 1.430861
Train Epoch: 12, 15/100, Loss: 0.101947
Train Epoch: 12, 16/100, Loss: 0.105148
Train Epoch: 12, 17/100, Loss: 0.045414
Train Epoch: 12, 18/100, Loss: 3.286052
Train Epoch: 12, 19/100, Loss: 1.281963
Train Epo

Train Epoch: 16, 83/100, Loss: 1.886510
Train Epoch: 16, 84/100, Loss: 5.556843
Train Epoch: 16, 85/100, Loss: 0.002635
Train Epoch: 16, 86/100, Loss: 0.002256
Train Epoch: 16, 87/100, Loss: 0.001041
Train Epoch: 16, 88/100, Loss: 2.492201
Train Epoch: 16, 89/100, Loss: 0.073139
Train Epoch: 16, 90/100, Loss: 1.735647
Train Epoch: 16, 91/100, Loss: 1.408645
Train Epoch: 16, 92/100, Loss: 0.855706
Train Epoch: 16, 93/100, Loss: 0.761600
Train Epoch: 16, 94/100, Loss: 0.000095
Train Epoch: 16, 95/100, Loss: 2.332014
Train Epoch: 16, 96/100, Loss: 1.460693
Train Epoch: 16, 97/100, Loss: 0.000482
Train Epoch: 16, 98/100, Loss: 0.009463
Train Epoch: 16, 99/100, Loss: 0.013883
Train Epoch: 18, 0/100, Loss: 0.026265
Train Epoch: 18, 1/100, Loss: 0.000001
Train Epoch: 18, 2/100, Loss: 0.001221
Train Epoch: 18, 3/100, Loss: 0.000003
Train Epoch: 18, 4/100, Loss: 0.320894
Train Epoch: 18, 5/100, Loss: 0.032547
Train Epoch: 18, 6/100, Loss: 2.116208
Train Epoch: 18, 7/100, Loss: 0.003384
Train Ep

...

Train Epoch: 576, 70/100, Loss: 2.504642
Train Epoch: 576, 71/100, Loss: 2.842662
Train Epoch: 576, 72/100, Loss: 2.034474
Train Epoch: 576, 73/100, Loss: 2.316774
Train Epoch: 576, 74/100, Loss: 2.466234
Train Epoch: 576, 75/100, Loss: 2.584091
Train Epoch: 576, 76/100, Loss: 2.025568
Train Epoch: 576, 77/100, Loss: 2.682530
Train Epoch: 576, 78/100, Loss: 2.080676
Train Epoch: 576, 79/100, Loss: 2.418191
Train Epoch: 576, 80/100, Loss: 2.415020
Train Epoch: 576, 81/100, Loss: 2.399345
Train Epoch: 576, 82/100, Loss: 2.365872
Train Epoch: 576, 83/100, Loss: 2.374953
Train Epoch: 576, 84/100, Loss: 2.325460
Train Epoch: 576, 85/100, Loss: 2.361391
Train Epoch: 576, 86/100, Loss: 2.351863
Train Epoch: 576, 87/100, Loss: 2.681823
Train Epoch: 576, 88/100, Loss: 2.304303
Train Epoch: 576, 89/100, Loss: 2.364333
Train Epoch: 576, 90/100, Loss: 2.663539
Train Epoch: 576, 91/100, Loss: 2.602679
Train Epoch: 576, 92/100, Loss: 2.051560
Train Epoch: 576, 93/100, Loss: 2.274568
Train Epoch: 576

Train Epoch: 582, 0/100, Loss: 2.331490
Train Epoch: 582, 1/100, Loss: 2.269465
Train Epoch: 582, 2/100, Loss: 2.131923
Train Epoch: 582, 3/100, Loss: 2.099498
Train Epoch: 582, 4/100, Loss: 2.116977
Train Epoch: 582, 5/100, Loss: 2.102744
Train Epoch: 582, 6/100, Loss: 2.456900
Train Epoch: 582, 7/100, Loss: 2.066413
Train Epoch: 582, 8/100, Loss: 2.521760
Train Epoch: 582, 9/100, Loss: 2.517199
Train Epoch: 582, 10/100, Loss: 2.503736
Train Epoch: 582, 11/100, Loss: 2.482290
Train Epoch: 582, 12/100, Loss: 2.453714
Train Epoch: 582, 13/100, Loss: 2.429868
Train Epoch: 582, 14/100, Loss: 2.008096
Train Epoch: 582, 15/100, Loss: 2.350083
Train Epoch: 582, 16/100, Loss: 2.525421
Train Epoch: 582, 17/100, Loss: 2.441372
Train Epoch: 582, 18/100, Loss: 2.303109
Train Epoch: 582, 19/100, Loss: 2.419079
Train Epoch: 582, 20/100, Loss: 2.245109
Train Epoch: 582, 21/100, Loss: 1.978668
Train Epoch: 582, 22/100, Loss: 2.446601
Train Epoch: 582, 23/100, Loss: 2.521326
Train Epoch: 582, 24/100, 

Train Epoch: 586, 70/100, Loss: 2.251275
Train Epoch: 586, 71/100, Loss: 2.307612
Train Epoch: 586, 72/100, Loss: 2.388135
Train Epoch: 586, 73/100, Loss: 2.384729
Train Epoch: 586, 74/100, Loss: 2.384545
Train Epoch: 586, 75/100, Loss: 2.362389
Train Epoch: 586, 76/100, Loss: 2.314819
Train Epoch: 586, 77/100, Loss: 2.328766
Train Epoch: 586, 78/100, Loss: 2.088285
Train Epoch: 586, 79/100, Loss: 2.363501
Train Epoch: 586, 80/100, Loss: 2.527212
Train Epoch: 586, 81/100, Loss: 2.298752
Train Epoch: 586, 82/100, Loss: 2.461075
Train Epoch: 586, 83/100, Loss: 2.281536
Train Epoch: 586, 84/100, Loss: 2.370824
Train Epoch: 586, 85/100, Loss: 2.222732
Train Epoch: 586, 86/100, Loss: 2.451539
Train Epoch: 586, 87/100, Loss: 2.232874
Train Epoch: 586, 88/100, Loss: 2.469776
Train Epoch: 586, 89/100, Loss: 2.181330
Train Epoch: 586, 90/100, Loss: 2.190747
Train Epoch: 586, 91/100, Loss: 2.153890
Train Epoch: 586, 92/100, Loss: 2.553682
Train Epoch: 586, 93/100, Loss: 2.345153
Train Epoch: 586

Train Epoch: 592, 0/100, Loss: 2.183829
Train Epoch: 592, 1/100, Loss: 2.446563
Train Epoch: 592, 2/100, Loss: 2.350840
Train Epoch: 592, 3/100, Loss: 2.259587
Train Epoch: 592, 4/100, Loss: 2.299427
Train Epoch: 592, 5/100, Loss: 2.253391
Train Epoch: 592, 6/100, Loss: 2.302202
Train Epoch: 592, 7/100, Loss: 2.142403
Train Epoch: 592, 8/100, Loss: 2.297811
Train Epoch: 592, 9/100, Loss: 2.283055
Train Epoch: 592, 10/100, Loss: 2.391295
Train Epoch: 592, 11/100, Loss: 2.237352
Train Epoch: 592, 12/100, Loss: 2.370300
Train Epoch: 592, 13/100, Loss: 2.256705
Train Epoch: 592, 14/100, Loss: 2.218000
Train Epoch: 592, 15/100, Loss: 2.547578
Train Epoch: 592, 16/100, Loss: 2.345790
Train Epoch: 592, 17/100, Loss: 2.264783
Train Epoch: 592, 18/100, Loss: 2.385522
Train Epoch: 592, 19/100, Loss: 2.409385
Train Epoch: 592, 20/100, Loss: 2.546326
Train Epoch: 592, 21/100, Loss: 2.169083
Train Epoch: 592, 22/100, Loss: 2.386416
Train Epoch: 592, 23/100, Loss: 2.378707
Train Epoch: 592, 24/100, 

Train Epoch: 596, 70/100, Loss: 2.334157
Train Epoch: 596, 71/100, Loss: 2.362135
Train Epoch: 596, 72/100, Loss: 2.306215
Train Epoch: 596, 73/100, Loss: 2.287245
Train Epoch: 596, 74/100, Loss: 2.318996
Train Epoch: 596, 75/100, Loss: 2.287670
Train Epoch: 596, 76/100, Loss: 2.273872
Train Epoch: 596, 77/100, Loss: 2.264068
Train Epoch: 596, 78/100, Loss: 2.407347
Train Epoch: 596, 79/100, Loss: 2.300222
Train Epoch: 596, 80/100, Loss: 2.374233
Train Epoch: 596, 81/100, Loss: 2.371606
Train Epoch: 596, 82/100, Loss: 2.206481
Train Epoch: 596, 83/100, Loss: 2.222707
Train Epoch: 596, 84/100, Loss: 2.187292
Train Epoch: 596, 85/100, Loss: 2.304225
Train Epoch: 596, 86/100, Loss: 2.402138
Train Epoch: 596, 87/100, Loss: 2.294662
Train Epoch: 596, 88/100, Loss: 2.397171
Train Epoch: 596, 89/100, Loss: 2.330009
Train Epoch: 596, 90/100, Loss: 2.380599
Train Epoch: 596, 91/100, Loss: 2.434834
Train Epoch: 596, 92/100, Loss: 2.364827
Train Epoch: 596, 93/100, Loss: 2.277813
Train Epoch: 596